------------------------------------------------------------------------------------------------------------------------------------------------------

## EVENT VISUALIZER NOTEBOOK

Make sure you have the data accesible (either mounted or running with the correct path in the input file)

##### PLEASE! When you are done with the tutorial, move to the ```notebooks``` folder to start your own analysis 
 In this way, you will be able to keep the tutorial as a reference for future use (or others to use it).

------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
import sys; sys.path.insert(0, '../'); from lib import *;
figure_features()

In [ ]:
# Set options for general visualitation
OPT  = {
    "MICRO_SEC":   True,                # Time in microseconds (True/False)
    "NORM":        False,               # Runs can be displayed normalised (True/False)
    "ALIGN":       True,                # Aligns waveforms in peaktime (True/False)
    "LOGY":        False,               # Runs can be displayed in logy (True/False)
    "SHOW_AVE":    "",                  # If computed, vis will show average (AveWvf,AveWvfSPE,etc.)
    "SHOW_PARAM":  False,               # Print terminal information (True/False)
    "CHARGE_KEY":  "ChargeAveRange",    # Select charge info to be displayed. Default: "ChargeAveRange" (if computed)
    "PEAK_FINDER": False,               # Finds possible peaks in the window (True/False)
    "LEGEND":      True,                # Shows plot legend (True/False)
    "SHOW":        True
    }

The following cell can be used to load the configuration file.
Check that everything is OK and choose your runs!

In [ ]:
info = read_input_file("TUTORIAL")      # Read input file
for key in info: print(key,info[key])  # Print loaded info
for key in info: print(key,info[key])  # Print loaded info

In [ ]:
chs  = [0,6]
runs = [1] 
my_runs = load_npy(np.asarray(runs).astype(int),np.asarray(chs).astype(int),preset="RAW",info=info,compressed=True) # LOAD YOUR RUNS

Once loaded you can see the loaded variables you have available to plot

In [ ]:
if check_key(my_runs[runs[0]][chs[0]], "UnitsDict") == False: get_units(my_runs)
for key in my_runs[runs[0]][chs[0]]: print(key)

### BASIC MODE

Plot a histogram with the loaded channels for each run. You need to select the event number to plot

In [ ]:
EVENT2PLOT = np.random.randint(len(my_runs[runs[0]][chs[0]]["RawADC"]))
print(EVENT2PLOT)

nch=len(chs); ncol=int((nch+1)/2); nrow=int((nch)/2); z=0; axs = []
if nch < 4: 
        fig, ax = plt.subplots(nch ,1, figsize = (10,8))
        axs = ax.T.flatten(); 
        if nch == 1: axs.append(ax)
        else: axs = ax
else: fig, ax = plt.subplots(dpi=200,ncols= ncol,nrows=nrow,figsize=[8,3],sharex=True)

time = np.arange(len(my_runs[runs[0]][chs[0]]["RawADC"][EVENT2PLOT]))*my_runs[runs[0]][chs[0]]["Sampling"]
for c,ch in enumerate(chs):
        i = int(z/ncol)
        j = z%ncol

        RawADC  = my_runs[runs[0]][ch]["RawADC"]    [EVENT2PLOT]
        PedSTD  = my_runs[runs[0]][ch]["RawPedSTD"] [EVENT2PLOT]
        PedMean = my_runs[runs[0]][ch]["RawPedMean"][EVENT2PLOT]
        ax[c].tick_params(axis='both', which='major', labelsize=5)
        ax[c].grid()
        ax[c].plot( time,  RawADC,linewidth=.5, label="Ch: "+str(ch))
        ax[c].plot([0, time[-1]], [PedMean,PedMean],                   color="k"  ,linewidth=.7)
        ax[c].plot([0, time[-1]], [PedMean+PedSTD,PedMean+PedSTD],"--",color="k"  ,linewidth=.7)
        ax[c].plot([0, time[-1]], [PedMean-PedSTD,PedMean-PedSTD],"--",color="k"  ,linewidth=.7)
        ax[c].legend()  
        ax[c].set_title("Run {} - Ch {} - Event Number {}".format(my_runs["NRun"][0],ch,EVENT2PLOT),size = 14)
        ax[c].xaxis.offsetText.set_fontsize(14) # Smaller fontsize for scientific notation
        ax[c].grid(True, alpha = 0.7)

        del RawADC,PedSTD,PedMean
        z+=1;   

fig.supylabel("ADC Counts")
fig.supxlabel(r'Time [s]')
# fig.supxlabel(r'Time [$\mu$s]')

plt.show()

### ADVANCED MODE

Use plotly to allow dynamical interaction.

In [ ]:
df = npy2df(my_runs)
# display(df)
wvf_df_index = df["RawADC"].index
# Make a new df from the exploded entries in wvf_df but conserve the index by looping over wvf_df_index
wvf_df = pd.DataFrame()
for i in wvf_df_index:
    print(i)
    index_array = df["RawADC"].loc[i[0],i[1]]
    index_df = pd.DataFrame(index_array).T
    index_df = index_df.assign(run=i[0], channel=i[1]).set_index(["run","channel"])
    index_df["tick"] = np.arange(0, index_df[0].size)
    # add this df to wvf_df with the index of the original df
    wvf_df = pd.concat([wvf_df, index_df], axis=0)
display(wvf_df)
var_df = df.drop(columns=["RawADC"])
display(var_df)

In [ ]:
def update_plot(n_clicks):
    fig = make_subplots(rows=1, cols=len(wvf_df.index.get_level_values("channel").unique()))
    if n_clicks >= 0: 
        for idx,i in enumerate(wvf_df.index.get_level_values("channel").unique()):
            # if raw:
            fig.add_trace(go.Scatter(line=dict(color="blue"),x=wvf_df.loc[(runs[0],i),"tick"], y=wvf_df.loc[(runs[0],i),n_clicks], name="Channel "+str(i)), row=1, col=idx+1)
            fig.add_trace(go.Scatter(mode="markers", marker=dict(color="red", size=10), x=[var_df.loc[(runs[0],i),"RawPeakTime"][n_clicks]], y=[var_df.loc[(runs[0],i),"RawPeakAmp"][n_clicks]]), row=1, col=idx+1)
            fig.add_hline(y=var_df.loc[(runs[0],i),"RawPedMean"][n_clicks], row=1, col=idx+1)
            fig.add_hline(y=var_df.loc[(runs[0],i),"RawPedMean"][n_clicks]+var_df.loc[(runs[0],i),"RawPedSTD"][n_clicks], row=1, col=idx+1)
            fig.add_hline(y=var_df.loc[(runs[0],i),"RawPedMean"][n_clicks]-var_df.loc[(runs[0],i),"RawPedSTD"][n_clicks], row=1, col=idx+1)
            custom_plotly_layout(fig,title='Raw Waveform - Event %i'%n_clicks, xaxis_title='Time [s]', yaxis_title='ADC')
            fig.update_layout(showlegend=False)
            # else:
            #     fig.add_trace(go.Scatter(line=dict(color="blue"),x=wvf_df.loc[(runs[0],i),"tick"], y=int(info["CHAN_POLAR"][idx])*(wvf_df.loc[(runs[0],i),0]-var_df.loc[(runs[0],i),"RawPedMean"][0]), name="Channel "+str(i)), row=1, col=idx+1)
            #     fig.add_trace(go.Scatter(mode="markers", marker=dict(color="red", size=10), x=[var_df.loc[(runs[0],i),"AnaPeakTime"][0]], y=[var_df.loc[(runs[0],i),"AnaPeakAmp"][0]]), row=1, col=idx+1)
            #     fig.update_layout(title='Raw Waveform - Event %i'%column, xaxis_title='Time [s]', yaxis_title='ADC',template="presentation")
        fig.show()
my_event = widgets.IntText(value=-1, description='#Event:')
widgets.interact(update_plot, n_clicks=my_event)